In [1]:
import numpy as np
import pandas as pd
import math

# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
pd.options.display.float_format = '{:.2f}'.format
# pd.options.display.max_rows = None  # 모든 row 내용 print
pd.options.display.max_rows = 20

In [3]:
# Provide the names for the columns since the CSV file with the data does not have a header row.
feature_names = ['symboling', 'normalized-losses', 'make', 'fuel-type',
        'aspiration', 'num-doors', 'body-style', 'drive-wheels',
        'engine-location', 'wheel-base', 'length', 'width', 'height', 'weight',
        'engine-type', 'num-cylinders', 'engine-size', 'fuel-system', 'bore',
        'stroke', 'compression-ratio', 'horsepower', 'peak-rpm', 'city-mpg',
        'highway-mpg', 'price']

In [4]:
# Load in the data from a CSV file that is comma separated.
car_data = pd.read_csv('https://storage.googleapis.com/mledu-datasets/cars_data.csv',
                     sep=',', names=feature_names, header=None, encoding='latin-1')

In [5]:
car_data = car_data.reindex(np.random.permutation(car_data.index))
# or equivalently,
# car_data = car_data.sample(frac=1)

In [6]:
print("Data set loaded. Num examples: ", len(car_data))

Data set loaded. Num examples:  205


In [7]:
car_data.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
151,1,87,toyota,gas,std,two,hatchback,fwd,front,95.70,...,92,2bbl,3.05,3.03,9.00,62,4800,31,38,6338
33,1,101,honda,gas,std,two,hatchback,fwd,front,93.70,...,92,1bbl,2.91,3.41,9.20,76,6000,30,34,6529
118,1,119,plymouth,gas,std,two,hatchback,fwd,front,93.70,...,90,2bbl,2.97,3.23,9.40,68,5500,37,41,5572
99,0,106,nissan,gas,std,four,hatchback,fwd,front,97.20,...,120,2bbl,3.33,3.47,8.50,97,5200,27,34,8949
93,1,103,nissan,gas,std,four,wagon,fwd,front,94.50,...,97,2bbl,3.15,3.29,9.40,69,5200,31,37,7349


In [8]:
LABEL = 'price'

numeric_feature_names = ['symboling', 'normalized-losses', 'wheel-base',
        'length', 'width', 'height', 'weight', 'engine-size', 'horsepower',
        'peak-rpm', 'city-mpg', 'highway-mpg', 'bore', 'stroke',
         'compression-ratio']

In [9]:
# Run to inspect numeric features.
car_data[numeric_feature_names]

,symboling,normalized-losses,wheel-base,length,width,height,weight,engine-size,horsepower,peak-rpm,city-mpg,highway-mpg,bore,stroke,compression-ratio
151,1,87,95.70,158.70,63.60,54.50,2040,92,62,4800,31,38,3.05,3.03,9.00
33,1,101,93.70,150.00,64.00,52.60,1940,92,76,6000,30,34,2.91,3.41,9.20
118,1,119,93.70,157.30,63.80,50.80,1918,90,68,5500,37,41,2.97,3.23,9.40
99,0,106,97.20,173.40,65.20,54.70,2324,120,97,5200,27,34,3.33,3.47,8.50
93,1,103,94.50,170.20,63.80,53.50,2024,97,69,5200,31,37,3.15,3.29,9.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,0,108,100.40,184.60,66.50,55.10,3060,181,152,5200,19,25,3.43,3.27,9.00
77,2,161,93.70,157.30,64.40,50.80,1944,92,68,5500,31,38,2.97,3.23,9.40
41,0,85,96.50,175.40,65.20,54.10,2465,110,101,5800,24,28,3.15,3.58,9.00
164,1,168,94.50,168.70,64.00,52.60,2204,98,70,4800,29,34,3.19,3.03,9.00


In [10]:
car_data[numeric_feature_names].describe()

,symboling,wheel-base,length,width,height,weight,engine-size,city-mpg,highway-mpg,compression-ratio
count,205.00,205.00,205.00,205.00,205.00,205.00,205.00,205.00,205.00,205.00
mean,0.83,98.76,174.05,65.91,53.72,2555.57,126.91,25.22,30.75,10.14
std,1.25,6.02,12.34,2.15,2.44,520.68,41.64,6.54,6.89,3.97
min,-2.00,86.60,141.10,60.30,47.80,1488.00,61.00,13.00,16.00,7.00
25%,0.00,94.50,166.30,64.10,52.00,2145.00,97.00,19.00,25.00,8.60
50%,1.00,97.00,173.20,65.50,54.10,2414.00,120.00,24.00,30.00,9.00
75%,2.00,102.40,183.10,66.90,55.50,2935.00,141.00,30.00,34.00,9.40
max,3.00,120.90,208.10,72.30,59.80,4066.00,326.00,49.00,54.00,23.00


In [11]:
# Coerce the numeric features to numbers. This is necessary because the model crashes because not all the values are numeric.
for feature_name in numeric_feature_names + [LABEL]:
    car_data[feature_name] = pd.to_numeric(car_data[feature_name], errors ='coerce')

"""
errors: error는 총 3개의 옵션이 존재합니다.
- errors = 'ignore' -> 만약 숫자로 변경할 수 없는 데이터라면 숫자로 변경하지 않고 원본 데이터를 그대로 반환합니다.
- errors = 'coerce' -> 만약 숫자로 변경할 수 없는 데이터라면 기존 데이터를 지우고 NaN으로 설정하여 반환합니다.
- errors = 'raise' -> 만약 숫자로 변경할 수 없는 데이터라면 에러를 일으키며 코드를 중단합니다."""
    
# Fill missing values with 0.
# Is this an OK thing to do? You may want to come back and revisit this decision later.
car_data.fillna(0, inplace=True)

In [12]:
car_data.shape

(205, 26)

In [13]:
X = car_data[numeric_feature_names].copy()
y = car_data['price'].copy()

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2 )

### normalization Zscore

In [15]:
car_data_norm = pd.DataFrame()
for feature_name in numeric_feature_names + [LABEL]:
    car_data_norm[feature_name] = (car_data[feature_name]-car_data[feature_name].mean())/car_data[feature_name].std()

In [16]:
car_data_norm.head()

,symboling,normalized-losses,wheel-base,length,width,height,weight,engine-size,horsepower,peak-rpm,city-mpg,highway-mpg,bore,stroke,compression-ratio,price
151,0.13,-0.18,-0.51,-1.24,-1.08,0.32,-0.99,-0.84,-1.01,-0.40,0.88,1.05,-0.40,-0.29,-0.29,-0.82
33,0.13,0.06,-0.84,-1.95,-0.89,-0.46,-1.18,-0.84,-0.67,1.33,0.73,0.47,-0.66,0.40,-0.24,-0.79
118,0.13,0.37,-0.84,-1.36,-0.98,-1.20,-1.22,-0.89,-0.86,0.61,1.80,1.49,-0.55,0.07,-0.19,-0.91
99,-0.67,0.14,-0.26,-0.05,-0.33,0.40,-0.44,-0.17,-0.15,0.18,0.27,0.47,0.12,0.51,-0.41,-0.50
93,0.13,0.09,-0.71,-0.31,-0.98,-0.09,-1.02,-0.72,-0.84,0.18,0.88,0.91,-0.21,0.18,-0.19,-0.69


In [17]:
X_norm = car_data_norm.copy()
y_norm = X_norm.pop('price')
X_train_norm, X_test_norm, y_train_norm, y_test_norm = train_test_split(X_norm, y_norm, test_size=0.2)

In [18]:
X_train_norm_dict = {name:np.array(value) for name, value in X_train_norm.items()}
y_train_norm_dict = {y_train_norm.name:y_train_norm.values}

In [19]:
feature_columns = []
feature_columns = [
    tf.feature_column.numeric_column(feature_name)
    for feature_name in ['symboling', 'width']
]

In [20]:
len(feature_columns)

2

In [21]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [22]:
X_train_dict = {name:np.array(value) for name, value in X_train.items()}

In [23]:
print(feature_layer(X_train_norm_dict))

tf.Tensor(
[[-0.669832    2.0474489 ]
 [-0.669832    1.1617521 ]
 [-1.472847    2.7000675 ]
 [ 0.13318297 -0.889335  ]
 [ 0.93619794  0.2760554 ]
 [-0.669832    0.92867404]
 [-2.275862    0.6023647 ]
 [ 0.13318297 -0.7961038 ]
 [ 1.7392129   0.2760554 ]
 [-2.275862    0.6023647 ]
 [ 0.93619794 -0.14348514]
 [ 0.13318297 -0.98256624]
 [ 1.7392129  -0.09686953]
 [ 0.93619794 -0.32994762]
 [-0.669832   -1.0757974 ]
 [ 1.7392129  -0.7961038 ]
 [ 0.13318297 -0.98256624]
 [ 0.13318297 -0.23671637]
 [-0.669832    0.4625179 ]
 [ 0.93619794 -0.7028725 ]
 [-0.669832   -1.0757974 ]
 [-0.669832    1.7211396 ]
 [ 1.7392129   0.92867404]
 [-0.669832    2.7000675 ]
 [ 1.7392129   1.1151365 ]
 [ 1.7392129   0.92867404]
 [-0.669832   -0.889335  ]
 [-1.472847    1.3482146 ]
 [ 0.93619794 -2.6141129 ]
 [-0.669832    1.7211396 ]
 [ 0.13318297 -0.7961038 ]
 [-0.669832   -0.51641005]
 [-0.669832   -0.32994762]
 [ 1.7392129   2.14068   ]
 [-0.669832   -0.32994762]
 [ 1.7392129   0.18282418]
 [-0.669832   -0.

In [24]:
feature_columns = []
feature_columns = [
    tf.feature_column.numeric_column(feature_name)
    for feature_name in numeric_feature_names
]
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

### convert dataframe to dict
when using feature_layer, <span style='color:red'> use dict </span> for input data in model.fit

In [25]:
X_train_norm_dict = {name:np.array(value) for name, value in X_train_norm.items()}
y_train_norm_dict = {y_train_norm.name:y_train_norm.values}

In [26]:
print(feature_layer(X_train_norm_dict))

tf.Tensor(
[[ 0.58890086 -0.49211898  2.8593526  ...  1.804244    1.319115
   2.0474489 ]
 [ 0.8130878   0.42501187  2.7334726  ...  1.3375468   1.5183917
   1.1617521 ]
 [ 0.3647139  -1.4092499  -0.4638766  ...  2.2747824   2.7970843
   2.7000675 ]
 ...
 [ 0.0844802   0.57786703 -0.36317268 ... -0.44089606  0.6050399
   0.2760554 ]
 [-0.13970675  0.11930157 -0.03588498 ... -0.49083075 -0.24188635
  -0.19010076]
 [ 0.30866715  0.8835773   2.9852324  ...  0.27739513  1.0201999
   0.08959294]], shape=(164, 15), dtype=float32)


In [27]:
model = tf.keras.models.Sequential()
model.add(feature_layer)
model.add(tf.keras.layers.Dense(units=3, input_shape=(1,), activation='relu'))
# model.add(tf.keras.layers.Dense(units=1, activation='relu')) 
#                         feature_layer일 경우 input_dim은 무조건 1:  바로 앞의 layer 수이므로 항상 1
model.add(tf.keras.layers.Dense(units=1))
model.compile(tf.keras.optimizers.Adam(learning_rate=0.01),
             loss='mean_squared_error',
             metrics=[tf.keras.metrics.MeanSquaredError()])

In [28]:
# X_train_norm_dict = {name:np.array(value) for name, value in car_data_norm.items()}
# # Caution: dict 만들때 주의
# y_train_norm_dict = np.array(X_train_norm_dict.pop('price'))

###  feature column사용시 <span style='color:red'> .fit 입력 주의 </span>
 feature column 사용시는 <span style='color:red'>  y value는 array만 사용</span>. x value는 dict 사용

In [29]:
batch_size=16
epochs=500
model.fit(X_train_norm_dict, y_train_norm_dict.values(), epochs=epochs, batch_size= batch_size)
# feature column 사용시는 y value는 array만 사용. x value는 dict 사용

Epoch 1/500
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
11/11 [==============================] - 2s 3ms/step - loss: 3.2761 - mean_squared_error: 3.2761
Epoch 2/500
11/11 [==============================] - 0s 4ms/step - loss: 1.4983 - mean_squared_error: 1.4983
Epoch 3/500
11/11 [==============================] - 0s 4ms/step - loss: 1.0394 - mean_squared_error: 1.0394
Epoch 4/500
11/11 [==============================] - 0s 5ms/step - loss: 0.9456 - mean_squared_error: 0.9456
Epoch 5/500
11/11 [==============================] - 0s 4ms/step - loss: 0.8856 - mean_squared_error: 0.8856
Epoch 6/500
11/11 [==============================] - 0s 4ms/step - loss: 0.8516 - mean_squared_error: 0.8516
Epoch 7/500
11/11 [==============================] - 0s 6ms/step - loss: 0.8259 - mean_squared_error: 0.8259
Epoch 8/500
11/11 [==============================] - 0s 9ms/step - loss: 0.8012 - mean_squared_error: 0.8012
Epoch 9/500
11/11 

11/11 [==============================] - 0s 4ms/step - loss: 0.2533 - mean_squared_error: 0.2533
Epoch 52/500
11/11 [==============================] - 0s 3ms/step - loss: 0.2528 - mean_squared_error: 0.2528
Epoch 53/500
11/11 [==============================] - 0s 4ms/step - loss: 0.2412 - mean_squared_error: 0.2412
Epoch 54/500
11/11 [==============================] - 0s 4ms/step - loss: 0.2423 - mean_squared_error: 0.2423
Epoch 55/500
11/11 [==============================] - 0s 8ms/step - loss: 0.2379 - mean_squared_error: 0.2379: 0s - loss: 0.2379 - mean_squared_error: 0.23
Epoch 56/500
11/11 [==============================] - 0s 7ms/step - loss: 0.2429 - mean_squared_error: 0.2429
Epoch 57/500
11/11 [==============================] - 0s 5ms/step - loss: 0.2362 - mean_squared_error: 0.2362
Epoch 58/500
11/11 [==============================] - 0s 5ms/step - loss: 0.2316 - mean_squared_error: 0.2316
Epoch 59/500
11/11 [==============================] - 0s 6ms/step - loss: 0.2264 - mean

11/11 [==============================] - 0s 3ms/step - loss: 0.1220 - mean_squared_error: 0.1220
Epoch 125/500
11/11 [==============================] - 0s 3ms/step - loss: 0.1297 - mean_squared_error: 0.1297
Epoch 126/500
11/11 [==============================] - 0s 3ms/step - loss: 0.1271 - mean_squared_error: 0.1271
Epoch 127/500
11/11 [==============================] - 0s 3ms/step - loss: 0.1222 - mean_squared_error: 0.1222
Epoch 128/500
11/11 [==============================] - 0s 3ms/step - loss: 0.1209 - mean_squared_error: 0.1209
Epoch 129/500
11/11 [==============================] - 0s 3ms/step - loss: 0.1192 - mean_squared_error: 0.1192
Epoch 130/500
11/11 [==============================] - 0s 5ms/step - loss: 0.1339 - mean_squared_error: 0.1339
Epoch 131/500
11/11 [==============================] - 0s 5ms/step - loss: 0.1261 - mean_squared_error: 0.1261
Epoch 132/500
11/11 [==============================] - 0s 4ms/step - loss: 0.1235 - mean_squared_error: 0.1235
Epoch 133/500
1

Epoch 197/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0943 - mean_squared_error: 0.0943
Epoch 198/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0971 - mean_squared_error: 0.0971
Epoch 199/500
11/11 [==============================] - 0s 4ms/step - loss: 0.1012 - mean_squared_error: 0.1012
Epoch 200/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0966 - mean_squared_error: 0.0966
Epoch 201/500
11/11 [==============================] - 0s 3ms/step - loss: 0.1001 - mean_squared_error: 0.1001
Epoch 202/500
11/11 [==============================] - 0s 3ms/step - loss: 0.1001 - mean_squared_error: 0.1001
Epoch 203/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0989 - mean_squared_error: 0.0989
Epoch 204/500
11/11 [==============================] - ETA: 0s - loss: 0.0577 - mean_squared_error: 0.05 - 0s 3ms/step - loss: 0.0940 - mean_squared_error: 0.0940
Epoch 205/500
11/11 [==============================] - 0s 3m

11/11 [==============================] - 0s 4ms/step - loss: 0.0945 - mean_squared_error: 0.0945
Epoch 270/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0898 - mean_squared_error: 0.0898
Epoch 271/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0905 - mean_squared_error: 0.0905
Epoch 272/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0824 - mean_squared_error: 0.0824
Epoch 273/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0788 - mean_squared_error: 0.0788
Epoch 274/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0855 - mean_squared_error: 0.0855
Epoch 275/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0867 - mean_squared_error: 0.0867
Epoch 276/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0821 - mean_squared_error: 0.0821
Epoch 277/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0821 - mean_squared_error: 0.0821
Epoch 278/500
1

11/11 [==============================] - 0s 4ms/step - loss: 0.0806 - mean_squared_error: 0.0806
Epoch 343/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0788 - mean_squared_error: 0.0788
Epoch 344/500
11/11 [==============================] - 0s 5ms/step - loss: 0.0816 - mean_squared_error: 0.0816
Epoch 345/500
11/11 [==============================] - 0s 6ms/step - loss: 0.0822 - mean_squared_error: 0.0822
Epoch 346/500
11/11 [==============================] - 0s 5ms/step - loss: 0.0750 - mean_squared_error: 0.0750
Epoch 347/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0784 - mean_squared_error: 0.0784
Epoch 348/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0783 - mean_squared_error: 0.0783
Epoch 349/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0819 - mean_squared_error: 0.0819
Epoch 350/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0823 - mean_squared_error: 0.0823
Epoch 351/500
1

11/11 [==============================] - 0s 5ms/step - loss: 0.0808 - mean_squared_error: 0.0808
Epoch 416/500
11/11 [==============================] - 0s 6ms/step - loss: 0.0834 - mean_squared_error: 0.0834
Epoch 417/500
11/11 [==============================] - 0s 5ms/step - loss: 0.0815 - mean_squared_error: 0.0815
Epoch 418/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0771 - mean_squared_error: 0.0771
Epoch 419/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0764 - mean_squared_error: 0.0764
Epoch 420/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0840 - mean_squared_error: 0.0840
Epoch 421/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0810 - mean_squared_error: 0.0810
Epoch 422/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0774 - mean_squared_error: 0.0774
Epoch 423/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0822 - mean_squared_error: 0.0822
Epoch 424/500
1

11/11 [==============================] - 0s 3ms/step - loss: 0.0774 - mean_squared_error: 0.0774
Epoch 489/500
11/11 [==============================] - 0s 5ms/step - loss: 0.0748 - mean_squared_error: 0.0748
Epoch 490/500
11/11 [==============================] - 0s 6ms/step - loss: 0.0752 - mean_squared_error: 0.0752
Epoch 491/500
11/11 [==============================] - 0s 7ms/step - loss: 0.0745 - mean_squared_error: 0.0745
Epoch 492/500
11/11 [==============================] - 0s 5ms/step - loss: 0.0748 - mean_squared_error: 0.0748
Epoch 493/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0749 - mean_squared_error: 0.0749
Epoch 494/500
11/11 [==============================] - 0s 5ms/step - loss: 0.0794 - mean_squared_error: 0.0794
Epoch 495/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0801 - mean_squared_error: 0.0801
Epoch 496/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0834 - mean_squared_error: 0.0834
Epoch 497/500
1

In [30]:
X_test_norm_dict = {name:np.array(value) for name, value in X_test_norm.items()}
y_test_norm_dict = {y_test_norm.name:y_test_norm.values}

In [31]:
model.evaluate(X_test_norm_dict, y_test_norm_dict.values())

Consider rewriting this model with the Functional API.
2/2 [==============================] - 1s 5ms/step - loss: 0.2017 - mean_squared_error: 0.2017


[0.20168639719486237, 0.20168639719486237]

### analysis using normalization without feature columns

In [32]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units=3, input_shape=(15,), activation='relu'))
model.add(tf.keras.layers.Dense(units=1))
model.compile(tf.keras.optimizers.Adam(learning_rate=0.01),
             loss='mean_squared_error',
             metrics=[tf.keras.metrics.MeanSquaredError()])

In [33]:
batch_size=16
epochs=500
model.fit(X_train_norm, y_train_norm, epochs=epochs, batch_size= batch_size)

Epoch 1/500
11/11 [==============================] - 1s 2ms/step - loss: 4.1056 - mean_squared_error: 4.1056
Epoch 2/500
11/11 [==============================] - 0s 3ms/step - loss: 1.5005 - mean_squared_error: 1.5005
Epoch 3/500
11/11 [==============================] - 0s 3ms/step - loss: 0.8627 - mean_squared_error: 0.8627
Epoch 4/500
11/11 [==============================] - 0s 2ms/step - loss: 0.7169 - mean_squared_error: 0.7169
Epoch 5/500
11/11 [==============================] - 0s 3ms/step - loss: 0.6222 - mean_squared_error: 0.6222
Epoch 6/500
11/11 [==============================] - 0s 3ms/step - loss: 0.5197 - mean_squared_error: 0.5197
Epoch 7/500
11/11 [==============================] - 0s 3ms/step - loss: 0.4353 - mean_squared_error: 0.4353
Epoch 8/500
11/11 [==============================] - 0s 3ms/step - loss: 0.3923 - mean_squared_error: 0.3923
Epoch 9/500
11/11 [==============================] - 0s 2ms/step - loss: 0.3596 - mean_squared_error: 0.3596
Epoch 10/500
11/11 

11/11 [==============================] - 0s 3ms/step - loss: 0.1179 - mean_squared_error: 0.1179
Epoch 76/500
11/11 [==============================] - 0s 3ms/step - loss: 0.1178 - mean_squared_error: 0.1178
Epoch 77/500
11/11 [==============================] - 0s 3ms/step - loss: 0.1172 - mean_squared_error: 0.1172
Epoch 78/500
11/11 [==============================] - 0s 3ms/step - loss: 0.1168 - mean_squared_error: 0.1168
Epoch 79/500
11/11 [==============================] - 0s 3ms/step - loss: 0.1153 - mean_squared_error: 0.1153
Epoch 80/500
11/11 [==============================] - 0s 3ms/step - loss: 0.1125 - mean_squared_error: 0.1125
Epoch 81/500
11/11 [==============================] - 0s 4ms/step - loss: 0.1156 - mean_squared_error: 0.1156
Epoch 82/500
11/11 [==============================] - 0s 3ms/step - loss: 0.1157 - mean_squared_error: 0.1157
Epoch 83/500
11/11 [==============================] - 0s 4ms/step - loss: 0.1135 - mean_squared_error: 0.1135
Epoch 84/500
11/11 [===

11/11 [==============================] - 0s 2ms/step - loss: 0.0905 - mean_squared_error: 0.0905
Epoch 150/500
11/11 [==============================] - 0s 2ms/step - loss: 0.1004 - mean_squared_error: 0.1004
Epoch 151/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0953 - mean_squared_error: 0.0953
Epoch 152/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0887 - mean_squared_error: 0.0887
Epoch 153/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0827 - mean_squared_error: 0.0827
Epoch 154/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0832 - mean_squared_error: 0.0832
Epoch 155/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0842 - mean_squared_error: 0.0842
Epoch 156/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0857 - mean_squared_error: 0.0857
Epoch 157/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0923 - mean_squared_error: 0.0923
Epoch 158/500
1

11/11 [==============================] - 0s 2ms/step - loss: 0.0826 - mean_squared_error: 0.0826
Epoch 223/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0850 - mean_squared_error: 0.0850
Epoch 224/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0804 - mean_squared_error: 0.0804
Epoch 225/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0757 - mean_squared_error: 0.0757
Epoch 226/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0767 - mean_squared_error: 0.0767
Epoch 227/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0756 - mean_squared_error: 0.0756
Epoch 228/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0754 - mean_squared_error: 0.0754
Epoch 229/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0752 - mean_squared_error: 0.0752
Epoch 230/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0809 - mean_squared_error: 0.0809
Epoch 231/500
1

11/11 [==============================] - 0s 2ms/step - loss: 0.0763 - mean_squared_error: 0.0763
Epoch 296/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0768 - mean_squared_error: 0.0768
Epoch 297/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0831 - mean_squared_error: 0.0831
Epoch 298/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0746 - mean_squared_error: 0.0746
Epoch 299/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0735 - mean_squared_error: 0.0735
Epoch 300/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0737 - mean_squared_error: 0.0737
Epoch 301/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0744 - mean_squared_error: 0.0744
Epoch 302/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0709 - mean_squared_error: 0.0709
Epoch 303/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0708 - mean_squared_error: 0.0708
Epoch 304/500
1

11/11 [==============================] - 0s 2ms/step - loss: 0.0717 - mean_squared_error: 0.0717
Epoch 369/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0737 - mean_squared_error: 0.0737
Epoch 370/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0722 - mean_squared_error: 0.0722
Epoch 371/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0725 - mean_squared_error: 0.0725
Epoch 372/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0694 - mean_squared_error: 0.0694
Epoch 373/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0699 - mean_squared_error: 0.0699
Epoch 374/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0762 - mean_squared_error: 0.0762
Epoch 375/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0762 - mean_squared_error: 0.0762
Epoch 376/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0739 - mean_squared_error: 0.0739
Epoch 377/500
1

11/11 [==============================] - 0s 2ms/step - loss: 0.0708 - mean_squared_error: 0.0708
Epoch 441/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0709 - mean_squared_error: 0.0709
Epoch 442/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0700 - mean_squared_error: 0.0700
Epoch 443/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0699 - mean_squared_error: 0.0699
Epoch 444/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0692 - mean_squared_error: 0.0692
Epoch 445/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0720 - mean_squared_error: 0.0720
Epoch 446/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0720 - mean_squared_error: 0.0720
Epoch 447/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0731 - mean_squared_error: 0.0731
Epoch 448/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0717 - mean_squared_error: 0.0717
Epoch 449/500
1

In [34]:
model.evaluate(X_test_norm, y_test_norm)

2/2 [==============================] - 0s 5ms/step - loss: 0.2113 - mean_squared_error: 0.2113


[0.21134082973003387, 0.21134082973003387]

### normalization using minmax with feature colums

In [35]:
car_data_norm = pd.DataFrame()
for feature_name in numeric_feature_names + [LABEL]:
    car_data_norm[feature_name] = (car_data[feature_name]-car_data[feature_name].min())/(car_data[feature_name].max()-car_data[feature_name].min()) 

In [37]:
X_norm = car_data_norm.copy()
y_norm = X_norm.pop('price')
X_train_norm, X_test_norm, y_train_norm, y_test_norm = train_test_split(X_norm, y_norm, test_size=0.2)

X_train_norm_dict = {name:np.array(value) for name, value in X_train_norm.items()}
y_train_norm_dict = {y_train_norm.name:y_train_norm.values}

feature_columns = []
feature_columns = [
    tf.feature_column.numeric_column(feature_name)
    for feature_name in numeric_feature_names
]
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

model = tf.keras.models.Sequential()
model.add(feature_layer)
model.add(tf.keras.layers.Dense(units=3, input_shape=(1,), activation='relu'))
# model.add(tf.keras.layers.Dense(units=1, activation='relu')) 
#                         feature_layer일 경우 input_dim은 무조건 1:  바로 앞의 layer 수이므로 항상 1
model.add(tf.keras.layers.Dense(units=1))
model.compile(tf.keras.optimizers.Adam(learning_rate=0.01),
             loss='mean_squared_error',
             metrics=[tf.keras.metrics.MeanSquaredError()])

batch_size=16
epochs=500
model.fit(X_train_norm_dict, y_train_norm_dict.values(), epochs=epochs, batch_size= batch_size)

Epoch 1/500
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
11/11 [==============================] - 2s 3ms/step - loss: 0.0226 - mean_squared_error: 0.0226
Epoch 2/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0142 - mean_squared_error: 0.0142
Epoch 3/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0117 - mean_squared_error: 0.0117
Epoch 4/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0104 - mean_squared_error: 0.0104
Epoch 5/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0099 - mean_squared_error: 0.0099
Epoch 6/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0087 - mean_squared_error: 0.0087
Epoch 7/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0085 - mean_squared_error: 0.0085
Epoch 8/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0078 - mean_squared_error: 0.0078
Epoch 9/500
11/11 

11/11 [==============================] - 0s 3ms/step - loss: 0.0042 - mean_squared_error: 0.0042
Epoch 53/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0047 - mean_squared_error: 0.0047
Epoch 54/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0044 - mean_squared_error: 0.0044
Epoch 55/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0041 - mean_squared_error: 0.0041
Epoch 56/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0048 - mean_squared_error: 0.0048
Epoch 57/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0041 - mean_squared_error: 0.0041
Epoch 58/500
11/11 [==============================] - 0s 5ms/step - loss: 0.0041 - mean_squared_error: 0.0041
Epoch 59/500
11/11 [==============================] - 0s 7ms/step - loss: 0.0043 - mean_squared_error: 0.0043
Epoch 60/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0044 - mean_squared_error: 0.0044
Epoch 61/500
11/11 [===

11/11 [==============================] - 0s 5ms/step - loss: 0.0039 - mean_squared_error: 0.0039
Epoch 126/500
11/11 [==============================] - 0s 6ms/step - loss: 0.0039 - mean_squared_error: 0.0039
Epoch 127/500
11/11 [==============================] - 0s 5ms/step - loss: 0.0037 - mean_squared_error: 0.0037
Epoch 128/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0038 - mean_squared_error: 0.0038
Epoch 129/500
11/11 [==============================] - 0s 5ms/step - loss: 0.0045 - mean_squared_error: 0.0045
Epoch 130/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0037 - mean_squared_error: 0.0037
Epoch 131/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0038 - mean_squared_error: 0.0038
Epoch 132/500
11/11 [==============================] - 0s 5ms/step - loss: 0.0039 - mean_squared_error: 0.0039
Epoch 133/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0035 - mean_squared_error: 0.0035
Epoch 134/500
1

11/11 [==============================] - 0s 5ms/step - loss: 0.0037 - mean_squared_error: 0.0037  
Epoch 198/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0035 - mean_squared_error: 0.0035
Epoch 199/500
11/11 [==============================] - 0s 6ms/step - loss: 0.0036 - mean_squared_error: 0.0036
Epoch 200/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0033 - mean_squared_error: 0.0033
Epoch 201/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0034 - mean_squared_error: 0.0034
Epoch 202/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0034 - mean_squared_error: 0.0034
Epoch 203/500
11/11 [==============================] - 0s 5ms/step - loss: 0.0038 - mean_squared_error: 0.0038
Epoch 204/500
11/11 [==============================] - 0s 6ms/step - loss: 0.0032 - mean_squared_error: 0.0032
Epoch 205/500
11/11 [==============================] - 0s 6ms/step - loss: 0.0034 - mean_squared_error: 0.0034
Epoch 206/500

11/11 [==============================] - 0s 3ms/step - loss: 0.0031 - mean_squared_error: 0.0031
Epoch 271/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0032 - mean_squared_error: 0.0032    
Epoch 272/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0034 - mean_squared_error: 0.0034
Epoch 273/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0033 - mean_squared_error: 0.0033
Epoch 274/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0032 - mean_squared_error: 0.0032
Epoch 275/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0031 - mean_squared_error: 0.0031
Epoch 276/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0031 - mean_squared_error: 0.0031    
Epoch 277/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0033 - mean_squared_error: 0.0033
Epoch 278/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0032 - mean_squared_error: 0.0032
Epoch 2

11/11 [==============================] - 0s 4ms/step - loss: 0.0031 - mean_squared_error: 0.0031
Epoch 344/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0036 - mean_squared_error: 0.0036
Epoch 345/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0031 - mean_squared_error: 0.0031
Epoch 346/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0034 - mean_squared_error: 0.0034
Epoch 347/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0031 - mean_squared_error: 0.0031
Epoch 348/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0041 - mean_squared_error: 0.0041
Epoch 349/500
11/11 [==============================] - 0s 5ms/step - loss: 0.0040 - mean_squared_error: 0.0040
Epoch 350/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0035 - mean_squared_error: 0.0035
Epoch 351/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0030 - mean_squared_error: 0.0030
Epoch 352/500
1

11/11 [==============================] - 0s 7ms/step - loss: 0.0035 - mean_squared_error: 0.0035
Epoch 416/500
11/11 [==============================] - 0s 6ms/step - loss: 0.0030 - mean_squared_error: 0.0030
Epoch 417/500
11/11 [==============================] - 0s 5ms/step - loss: 0.0029 - mean_squared_error: 0.0029
Epoch 418/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0033 - mean_squared_error: 0.0033
Epoch 419/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0034 - mean_squared_error: 0.0034
Epoch 420/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0035 - mean_squared_error: 0.0035
Epoch 421/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0047 - mean_squared_error: 0.0047
Epoch 422/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0037 - mean_squared_error: 0.0037
Epoch 423/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0034 - mean_squared_error: 0.0034
Epoch 424/500
1

11/11 [==============================] - 0s 4ms/step - loss: 0.0030 - mean_squared_error: 0.0030    
Epoch 488/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0031 - mean_squared_error: 0.0031
Epoch 489/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0027 - mean_squared_error: 0.0027
Epoch 490/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0031 - mean_squared_error: 0.0031
Epoch 491/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0042 - mean_squared_error: 0.0042
Epoch 492/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0028 - mean_squared_error: 0.0028
Epoch 493/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0030 - mean_squared_error: 0.0030
Epoch 494/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0048 - mean_squared_error: 0.0048
Epoch 495/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0034 - mean_squared_error: 0.0034
Epoch 496/5

In [38]:
X_test_norm_dict = {name:np.array(value) for name, value in X_test_norm.items()}
y_test_norm_dict = {y_test_norm.name:y_test_norm.values}

model.evaluate(X_test_norm_dict, y_test_norm_dict.values())

Consider rewriting this model with the Functional API.
2/2 [==============================] - 1s 5ms/step - loss: 0.0276 - mean_squared_error: 0.0276


[0.027588922530412674, 0.027588922530412674]

### normalization using minmax without feature colums

In [39]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units=3, input_shape=(15,), activation='relu'))
model.add(tf.keras.layers.Dense(units=1))
model.compile(tf.keras.optimizers.Adam(learning_rate=0.01),
             loss='mean_squared_error',
             metrics=[tf.keras.metrics.MeanSquaredError()])

batch_size=16
epochs=500
model.fit(X_train_norm, y_train_norm, epochs=epochs, batch_size= batch_size)

Epoch 1/500
11/11 [==============================] - 1s 2ms/step - loss: 0.0538 - mean_squared_error: 0.0538
Epoch 2/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0239 - mean_squared_error: 0.0239
Epoch 3/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0204 - mean_squared_error: 0.0204
Epoch 4/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0170 - mean_squared_error: 0.0170
Epoch 5/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0142 - mean_squared_error: 0.0142
Epoch 6/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0138 - mean_squared_error: 0.0138
Epoch 7/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0115 - mean_squared_error: 0.0115
Epoch 8/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0105 - mean_squared_error: 0.0105
Epoch 9/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0098 - mean_squared_error: 0.0098
Epoch 10/500
11/11 

11/11 [==============================] - 0s 3ms/step - loss: 0.0052 - mean_squared_error: 0.0052
Epoch 75/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0050 - mean_squared_error: 0.0050
Epoch 76/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0063 - mean_squared_error: 0.0063
Epoch 77/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0052 - mean_squared_error: 0.0052
Epoch 78/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0052 - mean_squared_error: 0.0052
Epoch 79/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0053 - mean_squared_error: 0.0053
Epoch 80/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0054 - mean_squared_error: 0.0054
Epoch 81/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0050 - mean_squared_error: 0.0050
Epoch 82/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0055 - mean_squared_error: 0.0055
Epoch 83/500
11/11 [===

11/11 [==============================] - 0s 2ms/step - loss: 0.0048 - mean_squared_error: 0.0048
Epoch 148/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0048 - mean_squared_error: 0.0048
Epoch 149/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0053 - mean_squared_error: 0.0053
Epoch 150/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0050 - mean_squared_error: 0.0050
Epoch 151/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0050 - mean_squared_error: 0.0050
Epoch 152/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0057 - mean_squared_error: 0.0057
Epoch 153/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0046 - mean_squared_error: 0.0046
Epoch 154/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0052 - mean_squared_error: 0.0052
Epoch 155/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0049 - mean_squared_error: 0.0049
Epoch 156/500
1

11/11 [==============================] - 0s 2ms/step - loss: 0.0052 - mean_squared_error: 0.0052
Epoch 221/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0047 - mean_squared_error: 0.0047
Epoch 222/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0051 - mean_squared_error: 0.0051
Epoch 223/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0047 - mean_squared_error: 0.0047
Epoch 224/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0048 - mean_squared_error: 0.0048
Epoch 225/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0050 - mean_squared_error: 0.0050
Epoch 226/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0048 - mean_squared_error: 0.0048
Epoch 227/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0046 - mean_squared_error: 0.0046
Epoch 228/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0047 - mean_squared_error: 0.0047
Epoch 229/500
1

11/11 [==============================] - 0s 2ms/step - loss: 0.0052 - mean_squared_error: 0.0052
Epoch 294/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0049 - mean_squared_error: 0.0049
Epoch 295/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0050 - mean_squared_error: 0.0050
Epoch 296/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0047 - mean_squared_error: 0.0047
Epoch 297/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0053 - mean_squared_error: 0.0053
Epoch 298/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0048 - mean_squared_error: 0.0048
Epoch 299/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0055 - mean_squared_error: 0.0055
Epoch 300/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0051 - mean_squared_error: 0.0051
Epoch 301/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0047 - mean_squared_error: 0.0047
Epoch 302/500
1

11/11 [==============================] - 0s 2ms/step - loss: 0.0050 - mean_squared_error: 0.0050
Epoch 367/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0045 - mean_squared_error: 0.0045
Epoch 368/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0046 - mean_squared_error: 0.0046
Epoch 369/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0047 - mean_squared_error: 0.0047
Epoch 370/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0055 - mean_squared_error: 0.0055
Epoch 371/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0051 - mean_squared_error: 0.0051
Epoch 372/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0047 - mean_squared_error: 0.0047
Epoch 373/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0047 - mean_squared_error: 0.0047
Epoch 374/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0046 - mean_squared_error: 0.0046
Epoch 375/500
1

Epoch 439/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0047 - mean_squared_error: 0.0047
Epoch 440/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0048 - mean_squared_error: 0.0048
Epoch 441/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0046 - mean_squared_error: 0.0046
Epoch 442/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0049 - mean_squared_error: 0.0049
Epoch 443/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0047 - mean_squared_error: 0.0047
Epoch 444/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0046 - mean_squared_error: 0.0046
Epoch 445/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0048 - mean_squared_error: 0.0048
Epoch 446/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0047 - mean_squared_error: 0.0047
Epoch 447/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0047 - mean_squared_error: 0.0047
E

In [40]:
model.evaluate(X_test_norm, y_test_norm)

2/2 [==============================] - 0s 4ms/step - loss: 0.0252 - mean_squared_error: 0.0252


[0.02521391585469246, 0.02521391585469246]